In [207]:
import sys
sys.path.append('./src')
import syllo_gen
import random
import json
from tqdm.autonotebook import tqdm
import openai
openai.api_key = open('key.txt').read().strip()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\cheny\AppData\Local\Temp\ipykernel_8676\2018132128.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# TODO: add existence 
# TODO: symmetry?
# TODO: grammar-check
# TODO: add adjective

In [193]:
q,v = syllo_gen.get_syllo(1)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_nouns(v)
print(q)
print(syllo_gen.question2fol(q))
print(syllo_gen.question2template(q, v, rand=False))

{'story': ['[R1]a[R2]', '[R0]a[R1]'], 'conclusion': '[R0]o[R2]'}
{'story': ['∀x ([R1](x) → [R2](x))', '∀x ([R0](x) → [R1](x))'], 'conclusion': '∃x ([R0](x) ∧ ¬[R2](x))'}
{'story': ['All bottle of water is government.', 'All cookie is bottle of water.'], 'conclusion': 'Some cookie is not government.'}


In [ ]:
all_possilbe = set()
prompt = ''
for j in range(100):
    if len(all_possilbe) == 24:
        break
    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)
    if not real:
        q = syllo_gen.negate_quesion(q)

    v = syllo_gen.random_assign_nouns(v)
    if tuple(q['story']) not in all_possilbe:
        all_possilbe.add((tuple(q['story'])))
        q = syllo_gen.question2template(q, v, rand=False)
        prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
        prompt += f'Answer: {real}\n\n'
print(prompt)

In [234]:
corret = 0
total= 100
for i in tqdm(range(total)):

    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)
    if not real:
        q = syllo_gen.negate_quesion(q)
    # v = syllo_gen.random_assign_nouns(v)
    q = syllo_gen.question2template(q, v, rand=True)
    tmp_prompt = prompt + f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}'

    tmp_prompt += "\nAnswer:"



    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=tmp_prompt,
    temperature=0,
    max_tokens=10,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
    )
    text = response['choices'][0]['text'].strip()
    if text not in ['True', 'False']:
        print(text)
        continue
    if text == 'True' and real:
        corret += 1
    elif text == 'False' and not real:
        corret += 1
    else:
        print(f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}')
        print(text)


print(corret/total)

  0%|          | 0/100 [00:00<?, ?it/s]

Story: All C are not B. A subset of B is A. Question: There is no A that is not C.
True
Story: C is always B. If something is A, then it is not B. Question: There is no A that is not C.
True
Story: There exists some B that is C. All things that are B are also A. Question: A is not C.
True
Story: Every C is not B. There is no A that is not B. Question: At lease one A is not C.
False
Story: All C is B. If something is A, then it is not B. Question: There is no A that is not C.
True
Story: There are B that are also C. Every instance of B is also an instance of A. Question: A is never C.
True
Story: B is always C. A subset of A is B. Question: A is not C for all A.
True
Story: Every instance of B is also an instance of C. Everything that is A is also B. Question: Sometimes A is not C.
True
Story: There is no B that is not C. All A is B. Question: Sometimes A is not C.
True
Story: Every B is C. All things that are B are also A. Question: A is never C.
True
Story: All C are not B. There is n

In [ ]:
# TODO: Add grounded statements